<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
import numpy as np

In [2]:
data = pd.read_csv("/datasets/insurance.csv")
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Данные выглядят нормально. Есть значения float, где должно быть int, но так даже лучше в нашей ситуации, так как нам всё-равно домнажать на float матрицу.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** нет

**Обоснование:** запишем формулу предсказания, подставив w: 
$$
1) a_1 = X(X^T X)^{-1} X^T y
$$
Дальше если домножить матрицу X на другую обратимую матрицу, то получим:
$$
2) a_2 = XP((XP)^T XP)^{-1} (XP)^T y
$$
Дальше будем работать с выражением 2) и докажем, что оно эквивалентно выражению 1):
$$
a_2 = XP((XP)^T XP)^{-1} (XP)^T y = XP((P^TX^TX)P)^{-1}(XP)^Ty = XPP^{-1}(P^T(X^TX))^{-1}P^TX^T y= XPP^{-1}(X^TX)(P^T)^{-1}P^TX^T y
$$
Сокращаем P-шки и получаем:
$$
a_2 = XPP^{-1}(X^TX)(P^T)^{-1}P^TX^T y = X(X^TX)X^T y = a_1
$$

In [5]:
features = data.drop("Страховые выплаты", axis = True)
features = features.values

target = data["Страховые выплаты"]
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size = .25, random_state = 1)

In [6]:
def check(train_features, test_features, train_target, test_target):
    model = LinearRegression()
    
    model.fit(train_features, train_target)
    predicted = model.predict(test_features)
    print("R2 score before encyption:",r2_score(test_target, predicted))

    P = np.random.uniform(size = (4,4))
    try:
        np.linalg.inv(P)
    except LinAlgError:
        return 0,0,0
    train_features_encrypted = train_features @ P
    test_features_encrypted = test_features @ P

    model.fit(train_features_encrypted, train_target)
    predicted = model.predict(test_features_encrypted)
    print("R2 score after encyption:",r2_score(test_target, predicted))
    return train_features_encrypted, test_features_encrypted, P

In [7]:
res = check(train_features, test_features, train_target, test_target)

R2 score before encyption: 0.3237732279242751
R2 score after encyption: 0.3237732279258819


## Алгоритм преобразования

**Алгоритм**

1. Создаём случайную матрицу P, со случайными значениями из однородного распределения с интервалом от 0 до единицы.
2. Проверяем получившуюся матрицу на обратимость, если обратима - идём дальше, иначе - пункт 1.(В нашем случае при получении в return 0,0,0 вызываем функцию снова)
2. Сохраняем матрицу P.
3. Домножаем матрицу признаков на матрицу P.
4. Сохраняем преобразованную матрицу.

**Обоснование**

Получившаяся матрица будет обратимой с вероятностью 0.999999... поэтому можем не делать проверку на обратимость. Также матрица задана случайно, поэтому не будет явных(всё-таки генераторов случайных чисел пока нет. Пользуемся генераторами псевдослучайных) закономерностей, которые помогли бы дешифровать матрицу обратно. Итого: по получившейся матрице нельзя найти закономерности, что делает задачу дешифровки без ключа практически невыполнимой. Даже если злоумышленник будет знать, что данные зашифрованны(что тоже нужно узнать), то легче пойти и найти человека с ключом, чем искать при перемножении каких матриц получится искомая, не зная ничего про эти матрицы, кроме количества столбцов в одной, и столбцов и строк в другой.

## Проверка алгоритма

In [8]:
res = check(train_features, test_features, train_target, test_target)

R2 score before encyption: 0.3237732279242751
R2 score after encyption: 0.32377322792472685


Тот же самый пример из пункта 2. Да, есть различие в метриках, но оно порядка 10^(-10), что скорее погрешность вычислений, а не реальная разница.